# Part 4: Agentic RAG with a Custom MCP Server

**[EN]** This notebook provides a unified setup guide for connecting Claude Desktop to your custom MCP server. It supports running the server both locally (on Windows/Mac/Linux) and remotely.
**[KR]**  이 노트북은 Claude Desktop을 자체 제작한 MCP 서버에 연결하기 위한 통합 설치 안내서입니다. 로컬(Windows/Mac/Linux) 및 원격 환경 모두에서 서버를 실행하는 시나리오를 지원합니다.

## Step 1: Prerequisites

**[EN]** Ensure you have Docker and Claude Desktop installed on your local machine.<br>
**[KR]** 로컬 머신에 Docker와 Claude Desktop이 설치되어 있는지 확인해주세요.

## Step 2: Run Server (if necessary) and Generate Configuration

**[EN]** Run the single cell below. It will guide you through the entire process: running the server locally if needed, and generating the correct configuration for your specific setup (Local/Remote, Windows/Mac/Linux).<br>
**[KR]** 아래의 단일 셀을 실행하세요. 이 셀이 전체 과정을 안내합니다: 필요시 서버를 로컬에서 실행하고, 현재 설정(로컬/원격, Windows/Mac/Linux)에 맞는 올바른 설정을 생성합니다.

In [ ]:
import os
import json
import platform

def parse_env_file(file_path):
    """Parses a .env file, removing quotes from values."""
    vars = {}
    try:
        with open(file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line and not line.startswith('#'):
                    key, value = line.split('=', 1)
                    # Remove quotes from the value
                    vars[key.strip()] = value.strip().strip('"'"'")
    except FileNotFoundError:
        return None
    return vars

server_location = ""
while server_location not in ["local", "remote"]:
    server_location = input("Is the MCP server running on your LOCAL machine or a REMOTE machine? (local/remote): ").lower()

REMOTE_SERVER_IP = ""

if server_location == 'local':
    print("--- Setting up local server ---")
    
    env_vars = parse_env_file('elastic.env')

    if not env_vars or "ES_URL" not in env_vars or "ES_API_KEY" not in env_vars:
        print("⚠️ Error: 'elastic.env' file not found or is missing variables.")
    else:
        es_url = env_vars['ES_URL']
        es_api_key = env_vars['ES_API_KEY']

        print("Building the main server Docker image... This might take a while.")
        os.system("docker build -t colpali-mcp-server ./colpali_mcp_server")
        
        docker_command = (
            f'docker run -d --rm --name colpali-mcp-server -p 8000:8000 '
            f'-e ES_URL={es_url} '
            f'-e ES_API_KEY={es_api_key} '
            f'colpali-mcp-server'
        )
        
        print("\nRunning the main server container in the background...")
        os.system(docker_command)
        print("\n✅ Local server is up and running!")

    os_type = platform.system()
    if os_type == "Windows" or os_type == "Darwin": # Darwin is macOS
        REMOTE_SERVER_IP = "host.docker.internal"
        print(f"\nDetected {os_type}, using '{REMOTE_SERVER_IP}' for local connection.")
    else: # Assuming Linux
        REMOTE_SERVER_IP = "172.17.0.1" # Default Docker bridge IP for host
        print(f"\nDetected {os_type}, using '{REMOTE_SERVER_IP}' for local connection.")
else: # remote
    REMOTE_SERVER_IP = input("Please enter the public IP address of your remote server: ")

config = {
    "mcpServers": {
        "colpali-search-server": {
            "command": "docker",
            "args": [
                "run", "-i", "--rm",
                "-e", f"REMOTE_URL=ws://{REMOTE_SERVER_IP}:8000",
                "ghcr.io/byungjoochoi/colpali/mcp-ws-proxy:latest"
            ]
        }
    }
}

print("\n✅ Configuration generated successfully!")
print("👇 Copy the JSON below and paste it into your local claude_desktop_config.json file:")
print("-" * 70)
print(json.dumps(config, indent=4))
print("-" * 70)

## Step 3: Final Steps

**[EN]** 
1. **Open your Claude Desktop configuration file.** You can find it at the following locations:
    - **macOS:** `~/Library/Application Support/Claude/claude_desktop_config.json`
    - **Windows:** `%APPDATA%\\Claude\\claude_desktop_config.json`

2. **Paste the configuration.** Copy the JSON generated above and paste it into the configuration file, replacing its contents.

3. **Restart Claude Desktop.** Claude will now have access to the tools provided by your server.

**[KR]** 
1. **Claude Desktop 설정 파일을 엽니다.** 다음 경로에서 찾을 수 있습니다:
    - **macOS:** `~/Library/Application Support/Claude/claude_desktop_config.json`
    - **Windows:** `%APPDATA%\\Claude\\claude_desktop_config.json`

2. **설정을 붙여넣습니다.** 위에서 생성된 JSON을 복사하여 설정 파일의 내용을 교체하세요.

3. **Claude Desktop을 재시작하세요.** 이제 Claude가 서버에서 제공하는 도구들을 사용할 수 있습니다.